In [1]:
import cv2
import os
import copy
import numpy as np

from ultis import *


In [2]:
#Đọc khung hình gốc của ảnh pano
origin_path = "D:/Xu ly anh/thucnghiem/v44"
origin_files = os.listdir(origin_path)
origin_files.sort()
origin_img = [cv2.imread(os.path.join(origin_path, img_file)) for img_file in origin_files]
#Đọc khung hình của video cần tracking
folder_path = "D:/Xu ly anh/thucnghiem/v47"
image_files = os.listdir(folder_path)
image_files.sort()
images = [cv2.imread(os.path.join(folder_path, img_file)) for img_file in image_files]
#Tạo đường dẫn lưu kết quả output
output_folder = "D:/Xu ly anh/thucnghiem/v44_compare_test"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [3]:
def compare(A, B, compare_method="akaze", output_csv="tracking_output.csv"):
    """
    A: danh sách các khung (array) của ảnh pano gốc
    B: danh sách các khung (array) của video cần tracking
    compare_method: thuật toán sử dụng để so sánh (akaze, sift, orb)
    output_csv: đường dẫn xuất ra file CSV chứa thông tin so sánh về các khung
    """
    with open(output_csv, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Frame_B", "Match_1", "Match_2", "Match_3"])

        for i in range(len(B)):
            scores = []
            img1 = cv2.cvtColor(B[i], cv2.COLOR_BGR2GRAY)

            for j in range(len(A)):
                img2 = cv2.cvtColor(A[j], cv2.COLOR_BGR2GRAY)
                
                if compare_method == "akaze":
                    similarity_percentage = calculate_similarity_percentage_akaze(img1, img2)
                elif compare_method == "sift":
                    similarity_percentage = calculate_similarity_percentage_sift(img1, img2)
                elif compare_method == "orb":
                    similarity_percentage = calculate_similarity_percentage_orb(img1, img2)
                else:
                    print("Không tìm được phương thức tính toán tương ứng")
                    return
                
                scores.append(similarity_percentage)

            sorted_indices = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
            top_3_indices = [index for index, _ in sorted_indices[:3]]
            writer.writerow([i] + top_3_indices)
        print("Compare list saved to " + output_csv)

        

In [4]:
method = "orb"
output_csv="tracking_output.csv"
A = origin_img
B = images


In [5]:
compare(A,B[:10],compare_method = method, output_csv = output_csv)

Compare list saved to tracking_output.csv


In [12]:
#Đọc ma trận H
csv_file = "D:/Xu ly anh/thucnghiem/homographies_DJI_0044_sift.csv"
H = read_homographies_from_csv(csv_file)
#Tính toán kết quả đầu ra
output_width, output_height, min_x, min_y = calculate_output_size(A, H)
output_shape = (output_width, output_height)
result_origin = cv2.imread("D:/Xu ly anh/thucnghiem/result_DJI_0044_sift.jpg")
print(output_shape)

(3082, 5851)


In [13]:
#Đọc file CSV
compare = []
with open(output_csv, mode='r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        compare.append([int(row['Match_1']), int(row['Match_2']), int(row['Match_3'])])


In [14]:
#Gắn màu cho 3 khung ảnh
compare_color = []
for i in range(len(compare)):
    top_3_values = compare[i]
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
    top_3_values = [(item, colors[i]) for i, item in enumerate(top_3_values)]
    compare_color.append(top_3_values)

In [15]:
#Xuất ra kết quả dưới dạng hình ảnh
for index, position in enumerate(compare_color):
    result = copy.deepcopy(result_origin)
    for pos, color in position:
        img = A[pos]
        img = draw_outer_bounding_box(img, color=color)
        warped_image = warp_image_to_zero(img, H, pos, output_shape, min_x, min_y)
        result = np.maximum(result, warped_image)

        frame_path = os.path.join(output_folder, f"frame_{index:04d}_compare.jpg")
        result_compare = concatenate_and_label_images(result, B[index], "result_compare", f"frame_{index:04d}.jpg")
        cv2.imwrite(frame_path, result_compare)


In [16]:
#Xuất video
folder_path = output_folder
output_path = "compare_video46_test.mp4"
frame_duration = 0.2  #thời gian xuất hiện 1 ảnh

create_video_from_frames(folder_path, output_path, frame_duration)